In [ ]:
import xarray as xr
import pandas as pd
from matplotlib import pyplot as plt
from holoviews import opts
import holoviews as hv
import panel as pn
from thredds_crawler.crawl import Crawl
import numpy as np
import datetime as dt

In [ ]:
hv.extension('bokeh')
pn.extension()

In [ ]:
def get_urls(start_date,end_date,start_time,model_name,site_name,variable):
    
    regex = ".*"+start_time+"\.nc"
    
    separator = ','
    select_vars = "?time," +variable
    
    separator = ''
    start_date_l = start_date.strftime("%Y-%m-%d").split('-')
    sd_concat = separator.join(start_date_l)
    end_date_l = end_date.strftime("%Y-%m-%d").split('-')
    ed_concat = separator.join(end_date_l)
    
    url = "https://thredds.met.no/thredds/catalog/alertness/YOPP_supersite/"+model_name+"/"+site_name+"/catalog.html"
    c = Crawl(url, select=[regex])
    
    bracket_urls =[]
    urls = []
    
    
    for d in c.datasets:
        for s in d.services:
            if s.get("service").lower() == "opendap":
                urls.append(s.get("url")+select_vars)
                if sd_concat in s.get("url") or ed_concat in s.get("url"):
                    bracket_urls.append(s.get("url")+select_vars)
   
    bracket_urls.sort()
    urls.sort()
    
    start_index = urls.index(bracket_urls[0])
    end_index = urls.index(bracket_urls[1])

    
    urls = urls[start_index:end_index+1]

    
    return urls

def day_sel(xarray):
    #default 24-48 leadtime from each model file
    start_time = xarray['time'][0].values+np.timedelta64((24),'h')
    end_time = start_time+np.timedelta64((24),'h')
    xarray = xarray.sel(time=slice(start_time,end_time))
    return xarray
    
    
#so far only model data
def get_data(out_type,urls):
   
    if out_type == 'concatenated':
        df=xr.open_mfdataset(urls,concat_dim=['time'],combine='nested',preprocess=day_sel).to_dataframe()
    else:
        df = xr.open_mfdataset(urls,concat_dim=['stime'],combine='nested').to_dataframe()
        df = df.unstack()
  
    return df


def plot_ts_stacked(data,variable):
    plots = []
    
    for item in data.columns.get_level_values(1):
        p = hv.Curve((data.index,data[variable][item])) 
        plots.append(p)
        
    plot = hv.Overlay(plots)

    return plot

def plot_ts_concat(data,variable):
    plot = hv.Curve((data.index,data[variable]))
    return plot

In [ ]:
types = pn.widgets.Select(
    name="types", options=['stacked','concatenated'], margin=(0, 20, 0, 0)
)
variable = pn.widgets.Select(
    name="Variables", options=['tas','snd'], margin=(0, 20, 0, 0)
)
site_name = pn.widgets.Select(
    name="Site", options=['sodankyla','tiksi'], margin=(0, 20, 0, 0)
)
model_name = pn.widgets.Select(
    name="model", options=['ifs-ecmwf','AROME-Arctic'], margin=(0, 20, 0, 0)
)

start_time = pn.widgets.Select(
    name="Start time", options=['00','12'], margin=(0, 20, 0, 0)
)

dates = pn.widgets.DateRangeSlider(
    name='Date Range Slider',
    start=dt.datetime(2018, 2, 1), end=dt.datetime(2018, 3, 31),
    value=(dt.datetime(2018, 2, 1), dt.datetime(2018, 3, 31))
)


In [ ]:
@pn.depends(types,variable,site_name,model_name,start_time,dates)

def get_plot(types,variable,site_name,model_name,start_time,dates):
    urls = get_urls(dates[0],dates[1],start_time,model_name,site_name,variable,)
    data = get_data(types,urls)
    if types == 'concatenated':
        plot = plot_ts_concat(data, variable)
    else:
        plot = plot_ts_stacked(data, variable)  
    plot.opts(title="Timevar "+variable+" "+model_name+" "+site_name,
              width=1200,
              height=400,
              ylabel=variable,
              xlabel="time: "+data.index[0].strftime("%Y-%-m-%d")+ " - " +data.index[-1].strftime("%Y-%-m-%d")
             )
    return plot

In [ ]:
pn.Column("timevar 1 sop 1", get_plot,pn.Row(pn.Column(site_name,model_name),pn.Column(variable,types),pn.Column(dates,start_time)),width_policy="max").servable()
#TO DO: add start time and end time user input